In [2]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
#!pip install catboost

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
myfolder = "../data/CMaps/"

# **Columns' names**

In [3]:
#Columns' names
'''
1)  unit number
2)	time, in cycles
3)	operational setting 1
4)	operational setting 2
5)	operational setting 3
6)	sensor measurement  1
7)	sensor measurement  2
...
26)	sensor measurement  21
'''
unitNames = ['UnitNumber']
timeCycles = ["TimeInCycles"]
operSets = ["OperSet"+str(i) for i in range(1,4)] # 1,2 et 3
sensorMes = ["SensorMes"+str(j) for j in range(1, 22)] # de 1 à 21
columnsNames = unitNames + timeCycles + operSets +sensorMes

# **Datasets loading**

In [4]:
def data_loading(x):
  train_path = myfolder + "train_"+ x +".txt"
  test_path = myfolder + "test_"+ x +".txt"
  rul_path = myfolder + "RUL_"+ x +".txt"
  train = pd.read_csv(train_path, delim_whitespace=True, names=columnsNames)
  test = pd.read_csv(test_path, delim_whitespace=True, names=columnsNames)
  rul = pd.read_csv(rul_path, delim_whitespace=True, names=["RUL_FD"])
  return train, test, rul

train_fd001, test_fd001, rul_fd001 = data_loading("FD001")
train_fd002, test_fd002, rul_fd002 = data_loading("FD002")
train_fd003, test_fd003, rul_fd003 = data_loading("FD003")
train_fd004, test_fd004, rul_fd004 = data_loading("FD004")

In [5]:
# Forcer l'affichage de toutes les colonnes
pd.set_option('display.max_columns', None)

train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.0,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.0,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.0,10.48,6.4213


# **RUL column generation for train and test set**

In [6]:
def rul_train_generation(x):
  rul = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  rul.columns = ['UnitNumber', 'max']
  x = x.merge(rul, on=['UnitNumber'], how='left')
  x['RUL'] = x['max'] - x['TimeInCycles']
  x.drop('max', axis=1, inplace=True)
  return x

train_fd001 = rul_train_generation(train_fd001)
train_fd002 = rul_train_generation(train_fd002)
train_fd003 = rul_train_generation(train_fd003)
train_fd004 = rul_train_generation(train_fd004)

In [7]:
train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.0,10.62,6.3670,320
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.0,24.37,14.6552,319
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.0,10.48,6.4213,318


In [8]:
def rul_test_generation(x, rul):

  rul["UnitNumber"] = rul.index + 1 # +1 pour que UnitNumber demarre de 1 au lieu de 0, car il s'agit du numero des moteur

  x = x.merge(rul, on=['UnitNumber'], how='left')

  max_cycle = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  max_cycle.columns = ['UnitNumber', 'max']
  x = x.merge(max_cycle, on=['UnitNumber'], how='left')
  x['RUL'] = x['RUL_FD'] + x['max'] - x['TimeInCycles']
  x.drop(['max', 'RUL_FD'], axis=1, inplace=True)

  return x

test_fd001 = rul_test_generation(test_fd001, rul_fd001)
test_fd002 = rul_test_generation(test_fd002, rul_fd002)
test_fd003 = rul_test_generation(test_fd003, rul_fd003)
test_fd004 = rul_test_generation(test_fd004, rul_fd004)

In [9]:
test_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,20.0072,0.700,100.0,491.19,606.67,1481.04,1227.81,9.35,13.60,332.52,2323.67,8704.98,1.07,43.83,313.03,2387.78,8048.98,9.2229,0.02,362,2324,100.00,24.31,14.7007,251
1,1,2,24.9984,0.620,60.0,462.54,536.22,1256.17,1031.48,7.05,9.00,174.46,1915.21,7999.94,0.93,36.11,163.61,2028.09,7863.46,10.8632,0.02,306,1915,84.93,14.36,8.5748,250
2,1,3,42.0000,0.842,100.0,445.00,549.23,1340.13,1105.88,3.91,5.69,137.34,2211.93,8305.38,1.01,41.52,129.98,2387.95,8071.13,9.3960,0.02,328,2212,100.00,10.39,6.4365,249


In [10]:
print("train_FD001: ",train_fd001.shape, ". test_FD001: ", test_fd001.shape,". RUL: ", rul_fd001.shape)
print("train_FD002: ",train_fd002.shape, ". test_FD002: ", test_fd002.shape,". RUL: ", rul_fd002.shape)
print("train_FD003: ",train_fd003.shape, ". test_FD003: ", test_fd003.shape,". RUL: ", rul_fd003.shape)
print("train_FD004: ",train_fd004.shape, ". test_FD004: ", test_fd004.shape,". RUL: ", rul_fd004.shape)

train_FD001:  (20631, 27) . test_FD001:  (13096, 27) . RUL:  (100, 2)
train_FD002:  (53759, 27) . test_FD002:  (33991, 27) . RUL:  (259, 2)
train_FD003:  (24720, 27) . test_FD003:  (16596, 27) . RUL:  (100, 2)
train_FD004:  (61249, 27) . test_FD004:  (41214, 27) . RUL:  (248, 2)


In [11]:
print("train_FD001: ",train_fd001.shape, ". test_FD001: ", test_fd001.shape,". nbr moteurs", len(train_fd001['UnitNumber'].unique()))
print("train_FD002: ",train_fd002.shape, ". test_FD002: ", test_fd002.shape,". nbr moteurs", len(train_fd002['UnitNumber'].unique()))
print("train_FD003: ",train_fd003.shape, ". test_FD003: ", test_fd003.shape,". nbr moteurs", len(train_fd003['UnitNumber'].unique()))
print("train_FD004: ",train_fd004.shape, ". test_FD004: ", test_fd004.shape,". nbr moteurs", len(train_fd004['UnitNumber'].unique()))

train_FD001:  (20631, 27) . test_FD001:  (13096, 27) . nbr moteurs 100
train_FD002:  (53759, 27) . test_FD002:  (33991, 27) . nbr moteurs 260
train_FD003:  (24720, 27) . test_FD003:  (16596, 27) . nbr moteurs 100
train_FD004:  (61249, 27) . test_FD004:  (41214, 27) . nbr moteurs 249


In [12]:
len(train_fd002['UnitNumber'].unique())

260

# **Data normalization**

In [13]:
def normalised_df(train, test):

  from sklearn.preprocessing import MinMaxScaler

  # Instancier l'objet MinMaxScaler pour normaliser les données
  scaler = MinMaxScaler()

  # Normaliser train
  train_scaled = scaler.fit_transform(train)
  train_df = pd.DataFrame(train_scaled)

  # Normaliser test
  test_scaled = scaler.fit_transform(test)
  test_df = pd.DataFrame(test_scaled)


  train_df.columns = train.columns
  test_df.columns = test.columns
    
  return train_df, test_df

In [14]:
train_fd001, test_fd001 = normalised_df(train_fd001, test_fd001)
train_fd002, test_fd002 = normalised_df(train_fd002, test_fd002)
train_fd003, test_fd003 = normalised_df(train_fd003, test_fd003)
train_fd004, test_fd004 = normalised_df(train_fd004, test_fd004)

In [15]:
train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,0.0,0.000000,0.999926,0.997625,1.0,0.000000,0.130347,0.272082,0.212586,0.000000,0.001882,0.002738,0.626983,0.269578,0.205128,0.458604,0.003593,0.993111,0.550773,0.400540,0.0,0.288660,0.627907,1.0,0.015473,0.015881,0.590406
1,0.0,0.001845,0.476147,0.831591,1.0,0.626985,0.647971,0.634407,0.511781,0.507937,0.498118,0.450787,0.862888,0.601411,0.358974,0.641234,0.450364,0.992395,0.481761,0.351346,0.0,0.608247,0.864693,1.0,0.477968,0.481487,0.588561
2,0.0,0.003690,0.999900,0.998694,1.0,0.000000,0.123646,0.271245,0.222481,0.000000,0.001255,0.004625,0.627110,0.265759,0.205128,0.456169,0.003202,0.993056,0.531031,0.423787,0.0,0.278351,0.627907,1.0,0.010764,0.018932,0.586716


# **Data splitting**

In [16]:
def data_split(train, test):

  # data split
  X_train = train.drop('RUL', axis=1)
  Y_train = train['RUL']
  X_test = test.drop('RUL', axis=1)
  Y_test = test['RUL']
    
  return X_train, Y_train, X_test, Y_test


# **Light Gradient Boosting on ElasticNet**

In [17]:
from sklearn.linear_model import ElasticNetCV
from lightgbm import LGBMRegressor

def my_lgb_elast(train, test):

    X_train, Y_train, X_test, Y_test = data_split(train, test)


    # Définition du modèle ElasticNet
    enet = ElasticNetCV(cv=5)

    # Ajustement du modèle aux données d'entraînement
    enet.fit(X_train, Y_train)

    # Meilleurs hyperparamètres trouvés
    print(enet.alpha_, enet.l1_ratio_)

    # Définition du modèle LightGBM
    lgbm = LGBMRegressor()

    # Définition des hyperparamètres à optimiser
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [50, 100, 200],
        'num_leaves': [20, 30, 40],
        'min_child_samples': [10, 20, 30]
    }


    # Recherche des hyperparamètres optimaux
    grid_search = GridSearchCV(lgbm, param_grid=param_grid, cv=5)
    grid_search.fit(X_train, Y_train)

    # Meilleurs hyperparamètres trouvés
    print("Meilleurs paramètres trouvés :")
    print(grid_search.best_params_)
    
    
    
    '''#-------------Train---------------------
    # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Train
    y_pred_train = grid_search.predict(X_train)
    # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
    mse_train = mean_squared_error(Y_train, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    mae_train = mean_absolute_error(Y_train, y_pred_train)
    mape_train = np.mean(np.abs((Y_train - y_pred_train) / Y_train)) * 100
    r2_train = r2_score(Y_train, y_pred_train)
    adjusted_r2_train = 1 - (1-r2_train)*(len(Y_train)-1)/(len(Y_train)-X_train.shape[1]-1)

    print('\n=============================Train=============================')
    print('MSE : ',mse_train * 100,'%')
    print('RMSE : ',rmse_train * 100,'%')
    print('MAE : ',mae_train * 100,'%')
    print('MAPE : ',mape_train,'%')
    print('R2 : ',r2_train * 100,'%')
    print('Adjusted R2 : ', adjusted_r2_train * 100,'%')'''

    #-------------Test---------------------
    # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Test
    y_pred_test = grid_search.predict(X_test)
    # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
    mse_test = mean_squared_error(Y_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(Y_test, y_pred_test)
    mape_test = np.mean(np.abs((Y_test - y_pred_test) / Y_test)) * 100
    r2_test = r2_score(Y_test, y_pred_test)
    adjusted_r2_test = 1 - (1-r2_test)*(len(Y_test)-1)/(len(Y_test)-X_test.shape[1]-1)

    print('\n=============================Test=============================')
    print('MSE : ',mse_test * 100,'%')
    print('RMSE : ',rmse_test * 100,'%')
    print('MAE : ',mae_test * 100,'%')
    print('MAPE : ',mape_test,'%')
    print('R2 : ',r2_test * 100,'%')
    print('Adjusted R2 : ', adjusted_r2_test * 100,'%')

In [18]:
my_lgb_elast(train_fd001, test_fd001)

0.0007598506125843051 0.5


KeyboardInterrupt: 

In [ ]:
my_lgb_elast(train_fd002, test_fd002)

In [ ]:
my_lgb_elast(train_fd003, test_fd003)

In [ ]:
my_lgb_elast(train_fd004, test_fd004)

In [ ]:
from sklearn.linear_model import ElasticNetCV
from lightgbm import LGBMRegressor

def my_lgb_elasticnet(train, test):

    X_train, Y_train, X_test, Y_test = data_split(train, test)


    # Définition du modèle ElasticNet
    enet = ElasticNetCV(cv=5)

    # Ajustement du modèle aux données d'entraînement
    enet.fit(X_train, Y_train)

    # Meilleurs hyperparamètres trouvés
    print(enet.alpha_, enet.l1_ratio_)

    # Définition du modèle LightGBM
    lgbm = LGBMRegressor()

    # Définition des hyperparamètres à optimiser
    param_grid = {
        'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1],
        'n_estimators': [50, 60, 70, 80, 90, 100, 110, 120, 130, 200],
        'num_leaves': [10, 15, 20, 25, 30, 35, 40, 45, 50, 60],
        'min_child_samples': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
    }


    # Recherche des hyperparamètres optimaux
    grid_search = GridSearchCV(lgbm, param_grid=param_grid, cv=5)
    grid_search.fit(X_train, Y_train)

    # Meilleurs hyperparamètres trouvés
    print("Meilleurs paramètres trouvés :")
    print(grid_search.best_params_)
    
    
    
    '''#-------------Train---------------------
    # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Train
    y_pred_train = grid_search.predict(X_train)
    # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
    mse_train = mean_squared_error(Y_train, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    mae_train = mean_absolute_error(Y_train, y_pred_train)
    mape_train = np.mean(np.abs((Y_train - y_pred_train) / Y_train)) * 100
    r2_train = r2_score(Y_train, y_pred_train)
    adjusted_r2_train = 1 - (1-r2_train)*(len(Y_train)-1)/(len(Y_train)-X_train.shape[1]-1)

    print('\n=============================Train=============================')
    print('MSE : ',mse_train * 100,'%')
    print('RMSE : ',rmse_train * 100,'%')
    print('MAE : ',mae_train * 100,'%')
    print('MAPE : ',mape_train,'%')
    print('R2 : ',r2_train * 100,'%')
    print('Adjusted R2 : ', adjusted_r2_train * 100,'%')'''

    #-------------Test---------------------
    # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Test
    y_pred_test = grid_search.predict(X_test)
    # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
    mse_test = mean_squared_error(Y_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(Y_test, y_pred_test)
    mape_test = np.mean(np.abs((Y_test - y_pred_test) / Y_test)) * 100
    r2_test = r2_score(Y_test, y_pred_test)
    adjusted_r2_test = 1 - (1-r2_test)*(len(Y_test)-1)/(len(Y_test)-X_test.shape[1]-1)

    print('\n=============================Test=============================')
    print('MSE : ',mse_test * 100,'%')
    print('RMSE : ',rmse_test * 100,'%')
    print('MAE : ',mae_test * 100,'%')
    print('MAPE : ',mape_test,'%')
    print('R2 : ',r2_test * 100,'%')
    print('Adjusted R2 : ', adjusted_r2_test * 100,'%')

In [ ]:
my_lgb_elasticnet(train_fd001, test_fd001)

In [ ]:
my_lgb_elasticnet(train_fd002, test_fd002)

In [ ]:
my_lgb_elasticnet(train_fd003, test_fd003)

In [ ]:
my_lgb_elasticnet(train_fd004, test_fd004)